# Rodando C++ com CPLEX através do makefile criado

In [ ]:
! make clean

In [ ]:
ls

In [ ]:
ls instancias/

In [ ]:
! make ti ARGS=instancias/instancia_10projetos_2sondas_delta_t28.dat

# Metaheurísticas

## Lendo dados

In [ ]:
def Read_data(path):
    
    import copy
    
    coords_x = []
    coords_y = []
    bacias = []
    nomes = []
    maturidades = []
    qualidades = []
    plays = []
    soterramentos = []
    pcgnas = []
    geracao = []
    migracao = []
    reservatorio = []
    geometria = []
    retencao = []
    pshc = []
    mc_vol = []
    mi_vol = []
    mc_vpl = []
    mi_vpl = []
    custos = []
    tempos_exec = []
    inicio_janela = []
    final_janela = []
    sondas_x = []
    sondas_y = []
    
    data = {
        1: coords_x,
        2: coords_y,
        3: bacias,
        4: nomes,
        5: maturidades,
        6: qualidades,
        7: plays,
        8: soterramentos,
        9: pcgnas,
        10: geracao,
        11: migracao,
        12: reservatorio,
        13: geometria,
        14: retencao,
        15: pshc,
        16: mc_vol,
        17: mi_vol,
        18: mc_vpl,
        19: mi_vpl,
        20: custos,
        21: tempos_exec,
        22: inicio_janela,
        23: final_janela,
        25: sondas_x,
        26: sondas_y
    }
    
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                n_projetos = int(line.split()[0])
                custo_total = float(line.split()[1])
                t_init = int(line.split()[2])
                t_final = int(line.split()[3])
                delta_t = int(line.split()[4])
                n_periodos = int(line.split()[5])
            elif i == 24:
                n_sondas = int(line.split()[0])
            elif i <= 26:
                for elem in line.split('[')[1].split(']')[0].split(','):
                    try:
                        data[i].append(float(elem))
                    except:
                        data[i].append(elem)
            elif i > 26:
                setup = []
                for k, l in enumerate(line.split('[[')[1].split('], [')):
                    setup.append([])
                    for elem in l.split(']]')[0].split(', '):
                        setup[k].append(float(elem))
    
    return n_projetos, n_sondas, n_periodos, t_init, t_final, delta_t, custo_total, data, setup


## Classe para lista duplamente encadeada

In [ ]:
class Node:
    
    """
    Documentação:
    """
    
    def __init__(self, data=None):
        self.data = data
        self.next = None
        self.prev = None
    

class DoublyLinkedList:
    
    """
    Documentação:
    """
    
    def __init__(self, nodes=None):
        self.head = None
        if nodes is not None:
            node = Node(data=nodes.pop(0))
            self.head = node
            for elem in nodes:
                new_node = Node(data=elem)
                node.next = new_node
                new_node.prev = node
                node = node.next
    
    def __iter__(self):
        node = self.head
        while node is not None:
            yield node
            node = node.next
    
    def add_node_begining(self, value_to_add):
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        new_node.next = self.head
        self.head.prev = new_node
        self.head = new_node
    
    def add_node_end(self, value_to_add):
        
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        for curr in self:
            pass
        curr.next = new_node
        new_node.prev = curr
    
    def add_node_after_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        new_node.next = node_ref.next
        new_node.prev = node_ref
        if (node_ref.next != None):
            node_ref.next.prev = new_node
        node_ref.next = new_node
    
    def add_node_before_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        if node_ref == self.head:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        else:
            node_ref.prev.next = new_node
            new_node.prev = node_ref.prev
            new_node.next = node_ref
            node_ref.prev = new_node
    
    def remove_node_by_ref(self, node):
        if node == self.head:
            if (node.next != None):
                node.next.prev = None
                self.head = node.next
            del node
        else:
            if (node.next != None):
                node.next.prev = node.prev
                node.prev.next = node.next
            else:
                node.prev.next = node.next
            del node
    
    def find_value(self, value):
        for node in self:
            if node.data == value: break
        return node

## Classe para Heap

In [ ]:
class MinHeap(object):
    
    def __init__(self, modo=None, array=None):
        if modo == None:
            self.dados = []
        if modo == 'array':
            self.Heapfy(array)
    
    def Ajustar_para_baixo(self, i):
        left_child = 2 * i + 1
        right_child = 2 * i + 2
        length = len(self.dados) - 1
        smallest = i
        if left_child <= length and self.dados[i] > self.dados[left_child]:
            smallest = left_child
        if right_child <= length and self.dados[smallest] > self.dados[right_child]:
            smallest = right_child
        if smallest != i:
            self.dados[i], self.dados[smallest] = self.dados[smallest], self.dados[i]
            self.Ajustar_para_baixo(smallest)
    
    def Ajustar_para_cima(self, i):
        parent = math.floor(i / 2)
        length = len(self.dados) - 1
        if parent >= 0 and self.dados[parent] > self.dados[i]:
            self.dados[i], self.dados[parent] = self.dados[parent], self.dados[i]
            self.Ajustar_para_cima(parent)
    
    def Heapfy(self, array):
        self.dados = array.copy()
        for i in reversed(range(len(array)//2)):
            self.Ajustar_para_baixo(i)
    
    def GetMin(self):
        return self.dados[0]
    
    def PopMin(self):
        self.dados[0], self.dados[-1] = self.dados[-1], self.dados[0]
        minimo = self.dados.pop()
        self.Ajustar_para_baixo(0)
        return minimo 
    
    def InsertItem(self, value):
        self.dados.append(value)
        self.Ajustar_para_cima(len(self.dados) - 1)
    
    def PopItem(self, index):
        self.dados[index], self.dados[-1] = self.dados[-1], self.dados[index]
        self.dados.pop()
        if index < len(self.dados):
            self.Ajustar_para_cima(index)
            self.Ajustar_para_baixo(0)
        return
    
    def HeapSort(self, array):
        self.dados = array.copy()
        self.Heapfy(self.dados)
        sorted_array = []
        for _ in range(len(self.dados)):
            self.dados[0], self.dados[-1] = self.dados[-1], self.dados[0]
            sorted_array.append(self.dados.pop())
            self.Ajustar_para_baixo(0)
        return sorted_array

## Classe para árvore de busca binária

In [ ]:
class NodeArvoreBinaria(object):
    """
    documentação da classe
    """
    def __init__(self, label, key, h_left=None, h_right=None):
        
        self.label = label
        self.key = key
        self.index = None
        self.parent = None
        self.left_child = None
        self.right_child = None
        if h_left != None: self.h_left = h_left
        if h_right != None: self.h_right = h_right
    
    def CopiarNode(self):
        key = self.GetKey()
        label = self.GetLabel()
        right_child = self.GetRightChild()
        left_child = self.GetLeftChild()
        parent = self.GetParent()
        
        novo_node = NodeArvoreBinaria(label, key)
        
        novo_node.right_child = right_child
        novo_node.left_child = left_child
        novo_node.parent = parent
        
        return novo_node
    
    def GetKey(self):
        if self == None: return None
        return self.key
    
    def SetKey(self, key):
        self.key = key
    
    def GetLabel(self):
        if self == None: return None
        return self.label
    
    def SetLabel(self, label):
        self.label = label
    
    def GetLeftChild(self):
        return self.left_child
    
    def SetLeftChild(self, child):
        self.left_child = child
    
    def GetRightChild(self):
        return self.right_child
    
    def SetRightChild(self, child):
        self.right_child = child
    
    def GetParent(self):
        return self.parent
    
    def SetParent(self, parent):
        self.parent = parent
    
    def Copiar_Dados(self, target):
        self.SetKey(target.GetKey())
        self.SetLabel(target.GetLabel())
    
    def Copiar_Filhos(self, target):
        self.SetLeftChild(target.GetLeftChild())
        self.SetRightChild(target.GetRightChild())
    
    def Deletar_Right_Child(self):
        filho = self.GetRightChild()
        self.right_child.SetParent(None)
        self.SetRightChild(None)
        del filho
    
    def Deletar_Left_Child(self):
        filho = self.GetLeftChild()
        self.left_child.SetParent(None)
        self.SetLeftChild(None)
        del filho
    

class Arvore_Binaria_de_Busca(object):
    """
    documentação da classe
    """
    def __init__(self):
        self.tamanho = 0
        self.root = None
    
    def GetRoot(self):
        return self.root
    
    def SetRoot(self, root):
        self.root = root
    
    def DeletarLeaf(self, node):
        if node.GetParent() == None:
            self.SetRoot(None)
            self.tamanho = 0
            del node
            return
        else:
            self.tamanho -= 1
            if node.GetKey() <= node.parent.GetKey():
                node.parent.Deletar_Left_Child()
            elif node.GetKey() > node.parent.GetKey():
                node.parent.Deletar_Right_Child()
            del node
    
    def DeletarPaiDeFilhoUnico(self, node):
        if node.GetKey() == self.root.GetKey():
            if node.GetLeftChild() != None:
                node.left_child.SetParent(None)
                self.SetRoot(node.left_child)
            elif node.GetRightChild() != None:
                node.right_child.SetParent(None)
                self.SetRoot(node.right_child)
        else:
            if node.GetKey() <= node.parent.GetKey():
                if node.GetLeftChild() != None:
                    node.left_child.SetParent(node.parent)
                    node.parent.SetLeftChild(node.left_child)
                elif node.GetRightChild() != None:
                    node.right_child.SetParent(node.parent)
                    node.parent.SetLeftChild(node.right_child)
            if node.GetKey() > node.parent.GetKey():
                if node.GetLeftChild() != None:
                    node.left_child.SetParent(node.parent)
                    node.parent.SetRightChild(node.left_child)
                elif node.GetRightChild() != None:
                    node.right_child.SetParent(node.parent)
                    node.parent.SetRightChild(node.right_child)
        del node
        self.tamanho -= 1
    
    def DeletarPaiDeDois(self, node, sucessor):
        if sucessor == node.right_child:
            sucessor.SetParent(node.parent)
            sucessor.SetLeftChild(node.left_child)
            node.left_child.SetParent(sucessor)
            
            if node.GetKey() == self.root.GetKey():
                self.SetRoot(sucessor)
            else:
                if node.GetKey() <= node.parent.GetKey():
                    node.parent.SetLeftChild(sucessor)
                elif node.GetKey() > node.parent.GetKey():
                    node.parent.SetRightChild(sucessor)
        else:
            if sucessor.GetRightChild() != None:
                sucessor.right_child.SetParent(sucessor.parent)
                sucessor.parent.SetLeftChild(sucessor.right_child)
            else:
                sucessor.parent.SetLeftChild(None)
            
            sucessor.SetParent(node.parent)
            sucessor.SetLeftChild(node.left_child)
            node.left_child.SetParent(sucessor)
            sucessor.SetRightChild(node.right_child)
            node.right_child.SetParent(sucessor)
            
            if node.GetKey() == self.root.GetKey():
                self.SetRoot(sucessor)
            else:
                if node.GetKey() <= node.parent.GetKey():
                    node.parent.SetLeftChild(sucessor)
                elif node.GetKey() > node.parent.GetKey():
                    node.parent.SetRightChild(sucessor)
        self.tamanho -= 1
        del node
    
    def Busca(self, valor, start='root'):
        
        # se nao for especificado o início da busca, começar pela raíz
        if start == 'root': start = self.GetRoot()
        
        # se não tiver raíz, retorna None
        if start == None:
            return 'arvore vazia'
        
        # se valor encotrado, retornar
        if start.GetKey() == valor:
            res_busca = {'found': True, 'node':start, 'key': start.GetKey(), 'label': start.GetLabel(), 'indice': start.index, 'parent': start.GetParent()}
            return res_busca
        
        # se valor não encontrado
        else:
            if valor < start.GetKey():
                start = start.GetLeftChild()
                if start == None:
                    res_busca = {'found': False, 'parent': start}
                    found = True
                    return res_busca
                else:
                    return self.Busca(valor, start=start)
            else:
                if start.GetRightChild() == None:
                    res_busca = {'found': False, 'parent': start}
                    found = True
                    return res_busca
                else:
                    start = start.GetRightChild()
                    return self.Busca(valor, start=start)
    
    def Inserir(self, label, key, start='root'):
        
        if start == 'root': start = self.root
        
        novo_node = NodeArvoreBinaria(label, key)
        
        # se a arvore está vazia
        if self.root == None:
            self.SetRoot(novo_node)
            novo_node.index = 0
            self.tamanho = 1
        # se a árvore já tem, pelo menos, a raíz
        else:
            if key <= start.GetKey():
                if start.GetLeftChild() == None:
                    start.SetLeftChild(novo_node)
                    novo_node.SetParent(start)
                    novo_node.index = start.index + 1
                    self.tamanho += 1
                else:
                    self.Inserir(label, key, start=start.left_child)
            elif key > start.GetKey():
                if start.GetRightChild() == None:
                    start.SetRightChild(novo_node)
                    novo_node.SetParent(start)
                    novo_node.index = start.index + 1
                    self.tamanho += 1
                else:
                    self.Inserir(label, key, start=start.right_child)
    
    def GetMin(self):
        if self.GetRoot() == None: return None
        start = self.root
        minimo = self.root.GetKey()
        min_node = start
        while start.GetLeftChild() != None:
            start = start.left_child
            if start.GetKey() < minimo:
                minimo = start.GetKey()
                min_node = start
        return min_node, minimo
    
    def GetMax(self):
        if self.GetRoot() == None: return None
        start = self.root
        maximo = self.root.GetKey()
        max_node = start
        while start.GetRightChild() != None:
            start = start.right_child
            if start.GetKey() > maximo:
                maximo = start.GetKey()
                max_node = start
        return max_node, maximo
    
    def GetSucessor(self, start):
        if start.GetRightChild() == None: return None
        sucessor = start.right_child
        while sucessor.GetLeftChild() != None:
            sucessor = sucessor.left_child
        return (sucessor, sucessor.GetKey())
    
    def Deletar(self, valor, start='root'):
        
        if start == 'root': start = self.root
        
        # se não achar valor
        if start == None: return start
        
        # se tem que ir pra esquerda
        if valor < start.GetKey():
            self.Deletar(valor, start=start.left_child)
        # se tem que ir pra direita
        elif valor > start.key:
            self.Deletar(valor, start.right_child)
        # se encontrou o valor, deletar
        else:
            # caso 1: se node já está numa folha, simplesmente remover
            if start.GetLeftChild() == None and start.GetRightChild() == None:
                self.DeletarLeaf(start)
                return
            
            # caso 2: se só tem um filho, filho substitui
            elif not(start.GetLeftChild() != None and start.GetRightChild() != None):
                self.DeletarPaiDeFilhoUnico(start)
                return
            
            # caso 3: se tem dois filhos, achar sucessor e substituir pelo node a ser deletado
            else:
                sucessor = self.GetSucessor(start)[0]
                self.DeletarPaiDeDois(start, sucessor)
                return
    
    def PopMin(self):
        min_node, minimo = self.GetMin()
        label = min_node.GetLabel()
        self.Deletar(minimo)
        return {'label': label, 'key': minimo}
    
    def PopMax(self):
        max_node, maximo = self.GetMax()
        label = max_node.GetLabel()
        self.Deletar(maximo)
        return {'label': label, 'key': maximo}

## Outras estruturas de dados

In [ ]:
import Fibonacci_Heap

In [ ]:
import Arvore_AVL

## Implementação da heurística de construção

In [ ]:
def PrintSolution(s):
    n_sondas = len(s.keys())
    print ('solução atual: ')
    for i in range(n_sondas):
        for node in s[i]:
            print ('sonda ', i, ' executa projetos: ', node.data)

def ConstruirRCL(heap=None, alpha=0.5):
    
    import copy
    import heapq
    
    heap_local = copy.copy(heap)
    
    rcl = []
    ord_data = []
    
    for i in range(len(heap)):
        ord_data.append(heapq.heappop(heap_local))
    
    s_min = ord_data[len(ord_data) - 1][0]
    s_max = ord_data[0][0]
    threshold = s_min + alpha * (s_max - s_min)
    
    for i in range(len(ord_data)):
        if (ord_data[i][0] <= threshold):
            rcl.append(ord_data[i][1])
    
    return rcl

def ConstruirSolucao(dados, n_projetos, n_sondas, n_periodos, custo_total, desloc, 
                     criterio='lucro/custo', alpha=0.5):
    
    import math
    import copy
    import numpy as np
    import heapq
    
    lag = n_sondas
    
    # inicializando solução
    s = {i:DoublyLinkedList(nodes=[[-1, 0, n_periodos]]) for i in range(n_sondas)}
    
    # inicializando lista de candidatos, por sonda
    s_candidatos = {i:[] for i in range(n_sondas)}
    
    # armazena lista de candidatos como uma heap, dependendo do critério guloso
    for i in range(n_sondas):
        for j in range(n_projetos):
            if (criterio == 'lucro'):
                criterio_val = dados[19][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'lucro/custo'):
                criterio_val = dados[19][j] / dados[20][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'lucro/duracao'):
                criterio_val = dados[19][j] / (dados[21][j]+1)
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'custo'):
                criterio_val = dados[20][j]
                heapq.heappush( s_candidatos[i], (criterio_val, j) )
            elif (criterio == 'duracao'):
                criterio_val = dados[21][j]
                heapq.heappush( s_candidatos[i], (criterio_val, j) )
            elif (criterio == 'janela'):
                criterio_val = dados[23][j] / dados[23][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
    
    # inicializa contador de gastos total
    gastos = 0
    
    # inicializa conjunto de projetos já selecionados
    proj_usados = set()
    
    # conjunto de sondas é representado com uma Heap
    sondas = [(-n_periodos, i) for i in range(n_sondas)]
    
    # inicializa valor da função objetivo da solução
    fitness = 0
    
    # enquanto nenhuma restrição for alcançada
    while (gastos < custo_total and sondas):
        
        # escolher uma sonda
        rcl = ConstruirRCL(heap=sondas, alpha=alpha)
        sonda = np.random.choice(rcl)
        tam_prev = [v for (v,k) in sondas if (k==sonda)][0]
        sondas.remove((tam_prev, sonda))
        heapq.heapify(sondas)
        
        # escolher um projeto
        rcl = ConstruirRCL(heap=s_candidatos[sonda], alpha=alpha)
        projeto = np.random.choice(rcl)
        val = [v for (v,k) in s_candidatos[sonda] if (k==projeto)][0]
        s_candidatos[sonda].remove((val, projeto))
        heapq.heapify(s_candidatos[sonda])
        
        # se valor do projeto não aumenta a função objetivo, pular
        if (dados[19][projeto] <= 0.):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        # se projeto já foi escolhido, pular
        if (projeto in proj_usados):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        # se executar o projeto excede limite de gastos, pular
        if (gastos + dados[20][projeto] > custo_total):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        aloc = False
        proc = dados[21][projeto]
        
        #print ('Tentando inserir projeto ', projeto, ' na sonda ', sonda)
        
        # percorre atividades da sonda para ver se existe janela disponível na sonda
        for i, node in enumerate(s[sonda]):
            
            # se node não for de janela de tempo (é de projeto)
            if (node.data[0] != -1):
                
                # verificar interceção entre projeto alocado e janela do projeto a alocar
                if ((node.data[1] <= dados[23][projeto]) and (node.data[2] >= dados[22][projeto])):
                    
                    # se posso chegar projeto alocado para trás
                    if (node.prev != None):
                        if (node.prev.data[0] == -1):
                            
                            flag_setup = None
                            delta_setup = 0
                            # tenho projeto alocado para frente? se sim, calcular mudança de desloc
                            if (node.next != None):
                                if (node.next.data[0] != -1):
                                    flag_setup = 'next'
                                    old_setup = desloc[node.data[0] + lag][node.next.data[0] + lag]
                                    new_setup = desloc[projeto + lag][node.next.data[0] + lag]
                                    delta_setup = old_setup - new_setup
                                elif (node.next.next != None):
                                    flag_setup = 'next next'
                                    old_setup = desloc[node.data[0] + lag][node.next.next.data[0] + lag]
                                    new_setup = desloc[projeto + lag][node.next.next.data[0] + lag]
                                    delta_setup = old_setup - new_setup
                            
                            # quanto seria o setup?
                            setup = desloc[node.data[0] + lag][projeto + lag]
                            
                            # quanto de espaço é preciso abrir?
                            if (flag_setup == 'next'):
                                delta_min = (setup + proc) - delta_setup
                            elif (flag_setup == 'next next'):
                                delta_min = (setup + proc) - (node.next.data[2]-node.next.data[1]+1) - delta_setup
                            else:
                                delta_min = (setup + proc)
                            
                            if (delta_min < 0):
                                delta_min = 9999
                            
                            # o espaço disponível na janela anterior basta?
                            if ( (node.prev.data[2] - node.prev.data[1] + 1) >= delta_min):
                                
                                # chegar o projeto alocado para trás é viável, pela sua janela de tempo?
                                if (dados[22][node.data[0]] <= node.data[1] - delta_min):
                                    
                                    # se não há mudança de desloc, ou a mudança é positiva
                                    if (delta_setup >= 0):
                                        
                                        # qual seria inicio e final
                                        inicio = node.data[2] - delta_min + 1
                                        final = inicio + setup + proc -1
                                        
                                        # se for viável inserir projeto nessa janela
                                        if ((inicio >= dados[22][projeto]) and (final <= dados[23][projeto])):
                                            # viável: fazer alterações
                                            
                                            #print ('Alocando projeto ', projeto, ' após chegar projeto', 
                                            #       node.data[0], ' para trás em ', delta_min, ' períodos', 
                                            #       ' sendo delta_setup=', delta_setup, ' inicio=', inicio, ' e final=', final)
                                            
                                            aloc = True
                                            node.data[1] -= delta_min
                                            node.data[2] -= delta_min
                                            
                                            if ( (node.prev.data[2] - node.prev.data[1] + 1) > delta_min):
                                                node.prev.data[2] -= delta_min
                                            else:
                                                s[sonda].remove_node_by_ref(node.prev)
                                            
                                            if (flag_setup == 'next'):
                                                node.next.data[1] += delta_setup
                                                s[sonda].add_node_after_node(value_to_add=[projeto, inicio, final], node_ref=node)
                                            elif (flag_setup == 'next next'):
                                                node.next.next.data[1] += delta_setup
                                                node.next.data[2] += delta_setup
                                                node.next.data[1] -= delta_min
                                                node.next.data[0] = projeto
                                            else:
                                                s[sonda].add_node_after_node(value_to_add=[projeto, inicio, final], node_ref=node)
                                            
                                            #PrintSolution(s)
                                            #VerificaRestricoes(s, dados, fitness, gastos, desloc)
                                    
                                    # senão, se o desloc aumentar, o próximo projeto continua viável
                                    else:
                                        flag_realoc = None
                                        # se aumentar pra trás
                                        if (flag_setup == 'next'):
                                            if (dados[22][node.next.data[0]] <= node.next.data[1] + delta_setup):
                                                flag_realoc = True
                                        elif (flag_setup == 'next next'):
                                            if (dados[22][node.next.next.data[0]] <= node.next.next.data[1] + delta_setup):
                                                flag_realoc = True
                                        
                                        if (flag_realoc):
                                            
                                            # qual seria inicio e final
                                            inicio = node.data[2] - delta_min + 1
                                            final = inicio + setup + proc -1
                                            
                                            # se for viável inserir projeto nessa janela
                                            if ((inicio >= dados[22][projeto]) and (final <= dados[23][projeto])):
                                                # viável: fazer alterações
                                                
                                                #print ('Alocando projeto ', projeto, ' após chegar projeto', 
                                                #       node.data[0], ' para trás em ', delta_min, ' períodos', 
                                                #       ' sendo delta_setup=', delta_setup, 
                                                #       ' deslocando próximo para trás', ' inicio=', inicio, ' e final=', final)
                                                
                                                aloc = True
                                                node.data[1] -= delta_min
                                                node.data[2] -= delta_min
                                                
                                                if ( (node.prev.data[2] - node.prev.data[1] + 1) > delta_min):
                                                    node.prev.data[2] -= delta_min
                                                else:
                                                    s[sonda].remove_node_by_ref(node.prev)
                                                
                                                if (flag_setup == 'next'):
                                                    node.next.data[1] += delta_setup
                                                    s[sonda].add_node_after_node(value_to_add=[projeto, inicio, final], node_ref=node)
                                                elif (flag_setup == 'next next'):
                                                    node.next.next.data[1] += delta_setup
                                                    node.next.data[2] += delta_setup
                                                    node.next.data[1] -= delta_min
                                                    node.next.data[0] = projeto
                                                
                                                #PrintSolution(s)
                                                #VerificaRestricoes(s, dados, fitness, gastos, desloc)
                                        
                                        # senão
                                        else:
                                            # se aumentar pra frente
                                            # testar janela
                                            if (flag_setup == 'next'):
                                                if (node.next.data[2] - delta_setup <= dados[23][node.next.data[0]]):
                                                    flag_realoc = True
                                            elif (flag_setup == 'next next'):
                                                if (node.next.next.data[2] - delta_setup <= dados[23][node.next.next.data[0]]):
                                                    flag_realoc = True
                                            
                                            if (flag_realoc):
                                                
                                                # testar conflito
                                                flag_conflict = None
                                                if (flag_setup == 'next'):
                                                    if (node.next.next != None):
                                                        if (node.next.next.data[0] == -1):
                                                            if (-delta_setup <= (node.next.next.data[2]-node.next.next.data[1]+1)):
                                                                flag_conflict = True
                                                
                                                if (flag_setup == 'next next'):
                                                    if (node.next.next.next != None):
                                                        if (node.next.next.next.data[0] == -1):
                                                            if (-delta_setup <= (node.next.next.next.data[2]-node.next.next.next.data[1]+1)):
                                                                flag_conflict = True
                                                
                                                if (flag_conflict):
                                                    
                                                    # qual seria o inicio e final
                                                    inicio = node.data[2] - (delta_min + delta_setup) + 1
                                                    final = inicio + setup + proc -1
                                                    
                                                    # se for viável inserir projeto nessa janela
                                                    if ((inicio >= dados[22][projeto]) and (final <= dados[23][projeto])):
                                                        # viável: fazer alterações
                                                        
                                                        #print ('Alocando projeto ', projeto, ' após chegar projeto', 
                                                        #       node.data[0], ' para trás em ', delta_min, ' períodos', 
                                                        #       ' sendo delta_setup=', delta_setup, 
                                                        #       ' deslocando próximo para frente', ' inicio=', inicio, ' e final=', final)
                                                        
                                                        aloc = True
                                                        node.data[1] -= (delta_min + delta_setup)
                                                        node.data[2] -= (delta_min + delta_setup)
                                                        
                                                        if ( (node.prev.data[2] - node.prev.data[1] + 1) > (delta_min + delta_setup)):
                                                            node.prev.data[2] -= (delta_min + delta_setup)
                                                        else:
                                                            s[sonda].remove_node_by_ref(node.prev)
                                                        
                                                        if (flag_setup == 'next'):
                                                            node.next.data[2] -= delta_setup
                                                            
                                                            if (-delta_setup < (node.next.next.data[2]-node.next.next.data[1]+1)):
                                                                node.next.next.data[1] -= delta_setup
                                                            else:
                                                                s[sonda].remove_node_by_ref(node.next.next)
                                                            
                                                            s[sonda].add_node_after_node(value_to_add=[projeto, inicio, final], node_ref=node)
                                                        elif (flag_setup == 'next next'):
                                                            node.next.next.data[2] -= delta_setup
                                                            
                                                            if (-delta_setup < (node.next.next.next.data[2]-node.next.next.next.data[1]+1)):
                                                                node.next.next.next.data[1] -= delta_setup
                                                            else:
                                                                s[sonda].remove_node_by_ref(node.next.next.next)
                                                            
                                                            node.next.data[1] -= (delta_min + delta_setup)
                                                            node.next.data[0] = projeto

                                                        #PrintSolution(s)
                                                        #VerificaRestricoes(s, dados, fitness, gastos, desloc)
                    
                    # se não conseguir chegar para trás
                    if (not aloc):
                        # se posso chegar projeto para frente
                        if (node.next != None):
                            if (node.next.data[0] == -1):
                                
                                # quanto muda de deslocamento?
                                delta_setup = 0
                                node_prev = copy.copy(node.prev)
                                while (node_prev != None):
                                    if (node_prev.data[0] != -1):
                                        old_setup = desloc[node_prev.data[0] + lag][node.data[0] + lag]
                                        new_setup = desloc[projeto + lag][node.data[0] + lag]
                                        delta_setup = old_setup - new_setup
                                        break
                                    else:
                                        node_prev = node_prev.prev
                                if (node_prev == None):
                                    old_setup = desloc[sonda][node.data[0] + lag]
                                    new_setup = desloc[projeto + lag][node.data[0] + lag]
                                    delta_setup = old_setup - new_setup
                                
                                # quanto seria o setup?
                                node_prev = copy.copy(node.prev)
                                while (node_prev != None):
                                    if (node_prev.data[0] != -1):
                                        setup = desloc[node_prev.data[0] + lag][projeto + lag]
                                        break
                                    else:
                                        node_prev = node_prev.prev
                                if (node_prev == None):
                                    setup = desloc[sonda][projeto + lag]
                                
                                # quanto de espaço é preciso abrir?
                                delta_min = (setup + proc)
                                
                                # o espaço disponível na janela seguinte basta?
                                if ((node.next.data[2] - node.next.data[1] + 1) >= (delta_min - delta_setup)):
                                    
                                    # se for viavel, pela janela de tempo, colocar projeto para frente
                                    if (node.data[2] + delta_min - delta_setup <= dados[23][node.data[0]]):
                                        
                                        # qual seria o inicio e o final
                                        final = node.data[1] + delta_min -1
                                        inicio = final - proc - setup + 1
                                        
                                        # se for viável inserir projeto nessa janela
                                        if ((inicio >= dados[22][projeto]) and (final <= dados[23][projeto])):
                                            
                                            #print ('Alocando projeto ', projeto, ' após chegar projeto', 
                                            #       node.data[0], ' para frente em ', delta_min, ' períodos', 
                                            #       ' sendo delta_setup=', delta_setup, ' inicio=', inicio, ' e final=', final)
                                            
                                            # viável: fazer alterações
                                            aloc = True
                                            node.data[1] += delta_min
                                            node.data[2] += delta_min - delta_setup
                                            
                                            if ((node.next.data[2] - node.next.data[1] + 1) > (delta_min - delta_setup)):
                                                node.next.data[1] += delta_min - delta_setup
                                            else:
                                                s[sonda].remove_node_by_ref(node.next)
                                            
                                            s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)

                                            #PrintSolution(s)
                                            #VerificaRestricoes(s, dados, fitness, gastos, desloc)
                                        
                if (aloc):

                    #print ('projeto ', projeto, ' sendo inserido (busca em nó de projeto) na sonda', sonda, 
                    #       ' de ', inicio, ' até ', final, ', sendo setup=', setup, ', e proc=', proc, ' delta_setup=', delta_setup)

                    proj_usados.add(copy.copy(projeto))
                    fitness += dados[19][projeto]
                    break
            
            # se node for de janela de tempo
            elif (node.data[0] == -1):
                # busca tempo de deslocamento / setup 
                # se node for head, considerar origem da sonda
                if node.prev == None:
                    prev = sonda
                # senão, considerar projeto anterior
                else:
                    prev = node.prev.data[0] + lag
                setup = desloc[prev][projeto + lag]
                
                # se existe projeto alocado após a janela atual, o setup para ele vai mudar quanto?
                if node.next != None:
                    old_setup = desloc[prev][node.next.data[0] + lag]
                    new_setup = desloc[projeto + lag][node.next.data[0] + lag]
                    delta_setup = old_setup - new_setup
                else:
                    delta_setup = 0
                
                # calcula o mínimo tempo de início e o máximo tempo de término do projeto dentro da janela
                release = max(node.data[1], dados[22][projeto])
                due = min(node.data[2] + delta_setup, dados[23][projeto])
                
                # se consigo colocar o projeto dentro da janela
                if (release + proc + setup -1 <= due):
                    
                    # escolher tempo de inicio para alocar projeto
                    delta1 = max((release - node.data[1]), (due - release) - (proc + setup) + (node.data[2] - due))
                    delta2 = max((node.data[2] - due), (due - release) - (proc + setup) + (release - node.data[1]))
                    if (delta1 >= delta2):
                        inicio = copy.copy(release)
                        final = copy.copy(release + setup + proc - 1)
                    else:
                        inicio = copy.copy(due - proc - setup + 1)
                        final = copy.copy(due)
                    
                    # atualiza release do next e due da janela, caso o setup tenha sido alterado
                    if delta_setup == 0:
                        
                        #print ('Alocando projeto ', projeto, ' na janela de tempo ', 
                        #       node.data[1], '-', node.data[2], ', de ', inicio, ' até ', final,
                        #       ' sendo delta_setup=', delta_setup)
                        
                        aloc = True
                    else:
                        # se for reduzir o deslocamento, não tem problema de viabilidade nem conflito
                        if (delta_setup > 0):
                            if (node.next.data[2] > node.next.data[1]):
                                
                                #print ('Alocando projeto ', projeto, ' na janela de tempo ', 
                                #       node.data[1], '-', node.data[2], ', de ', inicio, ' até ', final,
                                #       ' sendo delta_setup=', delta_setup)
                                
                                node.next.data[1] += delta_setup
                                node.data[2] += delta_setup
                                aloc = True
                        
                        # se for aumentar, tentar primeiro colocar diferença pra trás
                        elif (delta_setup < 0):
                            # posso reduzir a release do projeto que sofre modificação do deslocamento? 
                            # testar sua janela de tempo (viabilidade)
                            flag_tw = 'inviavel'
                            if (node.next.data[1] + delta_setup >= dados[22][node.next.data[0]]):
                                flag_tw = 'viavel'
                            flag_conflict = 'inviavel'
                            # testar conflito com projeto a ser inserido
                            if (node.next.data[1] + delta_setup > final):
                                flag_conflict = 'viavel'
                            
                            # ações para cada combinação de flag e marcar flag aloc
                            # se não tem problema de janela nem de conflito, segue o jogo
                            if ((flag_tw=='viavel') and (flag_conflict=='viavel')):
                                
                                #print ('Alocando projeto ', projeto, ' na janela de tempo ', 
                                #       node.data[1], '-', node.data[2], ', de ', inicio, ' até ', final,
                                #       ' sendo delta_setup=', delta_setup, ' aumentando para trás', 
                                #       ' sem conlito')
                                
                                node.next.data[1] += delta_setup
                                node.data[2] += delta_setup
                                aloc = True
                                
                            # senão, se não tem problema de janela, mas tem conflito
                            elif ((flag_tw=='viavel') and (flag_conflict=='inviavel')):
                                # teria que alterar inicio e final do projeto a ser inserido
                                # se continua dentro da janela 
                                if (inicio+delta_setup >= node.data[1]):
                                    # e é viável pela janela de tempo
                                    if (inicio+delta_setup >= dados[22][node.data[0]]):
                                        
                                        #print ('Alocando projeto ', projeto, ' na janela de tempo ', 
                                        #       node.data[1], '-', node.data[2], ', de ', inicio, ' até ', final,
                                        #       ' sendo delta_setup=', delta_setup, ' aumentando para trás', 
                                        #       ' com conlito parte 1')
                                        
                                        inicio += delta_setup
                                        final += delta_setup
                                        node.next.data[1] += delta_setup
                                        node.data[2] += delta_setup
                                        aloc = True
                                
                                # senão, se é viável pela janela de tempo
                                elif (inicio+delta_setup >= dados[22][node.data[0]]):
                                    # e consigo antecipar o prev
                                    if (node.prev != None):
                                        if (node.prev.prev != None):
                                            if (node.prev.prev.data[0] == -1):
                                                # quanto consigo antecipar o prev?
                                                delta_prev = node.prev.data[1] - max(node.prev.prev.data[1], dados[22][node.prev.data[0]])
                                                delta_prev = max(0, delta_prev)
                                                # se é suficiente
                                                if (delta_prev >= -delta_setup):
                                                    
                                                    #print ('Alocando projeto ', projeto, ' na janela de tempo ', 
                                                    #       node.data[1], '-', node.data[2], ', de ', inicio, ' até ', final,
                                                    #       ' sendo delta_setup=', delta_setup, ' aumentando para trás', 
                                                    #       ' com conlito parte 2')
                                                    
                                                    if (-delta_setup < (node.prev.prev.data[2]-node.prev.prev.data[1]+1)):
                                                        node.prev.prev.data[2] += delta_setup
                                                    else:
                                                        s[sonda].remove_node_by_ref(node.prev.prev)
                                                    
                                                    node.prev.data[1] += delta_setup
                                                    node.prev.data[2] += delta_setup
                                                    
                                                    inicio += delta_setup
                                                    final += delta_setup
                                                    node.next.data[1] += delta_setup
                                                    node.data[2] += delta_setup
                                                    aloc = True
                            
                            # se ainda não foi alocado, tentar colocar diferença para frente
                            if (not aloc):
                                # posso aumentar a due do projeto que sofre modificação do deslocamento?
                                # testar sua janela de tempo (viabilidade)
                                flag_tw = 'inviavel'
                                if (node.next.data[2] - delta_setup <= dados[23][node.next.data[0]]):
                                    flag_tw = 'viavel'
                                # testar conflito com projeto next
                                flag_conflict = 'inviavel'
                                if (node.next.next != None):
                                    if (node.next.next.data[0] == -1):
                                        if (-delta_setup <= (node.next.next.data[2]-node.next.next.data[1]+1)):
                                            flag_conflict = 'viavel'
                                
                                # ações para cada combinação de flag e marcar flag aloc
                                # se não tem problema de janela nem conflito, segue o jogo
                                if ((flag_tw == 'viavel') and (flag_conflict == 'viavel')):
                                    
                                    #print ('Alocando projeto ', projeto, ' na janela de tempo ', 
                                    #       node.data[1], '-', node.data[2], ', de ', inicio, ' até ', final,
                                    #       ' sendo delta_setup=', delta_setup, ' aumentando para frente', 
                                    #       ' sem conlito')
                                    
                                    node.next.data[2] -= delta_setup
                                    if (node.next.next != None):
                                        if (-delta_setup < (node.next.next.data[2]-node.next.next.data[1]+1)):
                                            node.next.next.data[1] -= delta_setup
                                        else:
                                            s[sonda].remove_node_by_ref(node.next.next)
                                    aloc = True
                                    
                                # senão, se não tem problema de janela, mas tem de conflito
                                elif ((flag_tw == 'viavel') and (flag_conflict == 'inviavel')):
                                    # teria que tentar postergar projeto next next
                                    if (node.next.next.next != None):
                                        if (node.next.next.next.data[0] == -1):
                                            if (-delta_setup <= (node.next.next.next.data[2]-node.next.next.next.data[1]+1)):
                                                if (node.next.next.data[2] - delta_setup <= dados[23][node.next.next.data[0]]):
                                                    
                                                    #print ('Alocando projeto ', projeto, ' na janela de tempo ', 
                                                    #       node.data[1], '-', node.data[2], ', de ', inicio, ' até ', final,
                                                    #       ' sendo delta_setup=', delta_setup, ' aumentando para frente', 
                                                    #       ' com conlito')
                                                    
                                                    if (-delta_setup < (node.next.next.next.data[2]-node.next.next.next.data[1]+1)):
                                                        node.next.next.next.data[1] -= delta_setup
                                                    else:
                                                        s[sonda].remove_node_by_ref(node.next.next.next)
                                                    
                                                    node.next.next.data[2] -= delta_setup
                                                    node.next.next.data[1] -= delta_setup
                                                    node.next.data[2] -= delta_setup
                                                    aloc = True
                    
                    if (aloc):
                        
                        #print ('projeto ', projeto, ' sendo inserido na sonda', sonda, ' de ', inicio, ' até ', final,
                        #      ', sendo setup=', setup, ', e proc=', proc, ' delta_setup=', delta_setup)
                        
                        # alocar
                        proj_usados.add(copy.copy(projeto))
                        fitness += dados[19][projeto]
                        
                        # atualizando a lista de atividades ========================================
                        # se projeto preenche janela toda
                        if ( (int(node.data[1])==int(inicio)) and (int(node.data[2])==int(final)) ):
                        
                            # substituir janela pela alocação do projeto
                            node.data = [copy.copy(projeto), inicio, final]
                        
                        # senão, se projeto preenche início da janela    
                        elif (int(node.data[1])==int(inicio)):
                            
                            # atualizar release da janela
                            node.data[1] = final + 1
                            
                            # inserir projeto antes da janela
                            s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                            
                        # senão, se projeto preenche final da janela
                        elif (int(node.data[2])==int(final)):
                            
                            # atualiza a due da janela
                            node.data[2] = inicio - 1
                            
                            # inserir projeto depois da janela
                            s[sonda].add_node_after_node(value_to_add=[projeto, inicio, final], node_ref=node)
                        
                        # senão, se projeto preenche o meio da janela
                        else:
                            
                            # atualiza release da janela
                            temp = copy.copy(node.data[1])
                            node.data[1] = final + 1
                            
                            # insere janela nova antes da janela original
                            s[sonda].add_node_before_node(value_to_add=[-1, temp, inicio - 1], node_ref=node)
                            
                            # insere projeto antes da janela original
                            s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                        
                        #PrintSolution(s)
                        #VerificaRestricoes(s, dados, fitness, gastos, desloc)
                        
                        break
                else:
                    # senão, tentar realocar projetos ====================================================
                    
                    # verificar se janela da sonda tem interceção com janela do projeto
                    if ((node.data[1] <= dados[23][projeto]) and (node.data[2] >= dados[22][projeto])):
                        
                        # consigo antecipar o prev? quanto?
                        delta_prev = 0
                        if node.prev == None:
                            delta_prev = 0
                        elif node.prev.prev == None:
                            delta_prev = 0
                        else:
                            if node.prev.prev.data[0] != -1:
                                delta_prev = 0
                            else:
                                delta_prev = node.prev.data[1] - max(node.prev.prev.data[1], dados[22][node.prev.data[0]])
                                delta_prev = max(0, delta_prev)
                        
                        # consigo postergar o next? quanto?
                        delta_next = 0
                        if node.next == None:
                            delta_next = 0
                        elif node.next.next == None:
                            delta_next = 0
                        else:
                            if node.next.next.data[0] != -1:
                                delta_next = 0
                            else:
                                delta_next = min(node.next.next.data[2], dados[23][node.next.data[0]]) - node.next.data[2]
                                delta_next = max(0, delta_next)
                        
                        # quanto de delta eu preciso?
                        delta_min = (setup + proc) - (node.data[2] - node.data[1] + 1) - delta_setup

                        # só com antecipação do prev, torna viável?
                        if (delta_prev >= delta_min):
                            # se contempla janela
                            if ((node.data[1]-delta_min>=dados[22][projeto]) and (node.data[2]+delta_setup<=dados[23][projeto])):
                                flag_realoc = 'viavel'
                                # se há mudança de setup para mais
                                if (delta_setup < 0):
                                    # checar viabilidade de diminuir release do next
                                    if (node.next.data[1] + delta_setup < dados[22][node.next.data[0]]):
                                        flag_realoc = 'inviavel'
                                if (flag_realoc == 'viavel'):
                                    # fazer antecipação do prev
                                    node.prev.data[1] -= delta_min
                                    node.prev.data[2] -= delta_min
                                    
                                    if (node.prev.prev.data[2] - delta_min < node.prev.prev.data[1]):
                                        s[sonda].remove_node_by_ref(node.prev.prev)
                                    else:
                                        node.prev.prev.data[2] -= delta_min
                                    
                                    aloc = True
                                    node.data[1] -= delta_min

                        # só com postergação do next, torna viável?
                        elif (delta_next >= delta_min):
                            # se contempla janela
                            if ((node.data[1]>=dados[22][projeto]) and (node.data[2]+delta_min+delta_setup<=dados[23][projeto])):
                                flag_realoc = 'viavel'
                                # se há mudança de setup para mais
                                if (delta_setup < 0):
                                    # checar viabilidade de diminuir release do next
                                    if (node.next.data[1] + delta_setup < dados[22][node.next.data[0]]):
                                        flag_realoc = 'inviavel'
                                if (flag_realoc == 'viavel'):
                                    # fazer postergação do next
                                    node.next.data[1] += delta_min
                                    node.next.data[2] += delta_min
                                    
                                    if (node.next.next.data[1] + delta_min > node.next.next.data[2]):
                                        s[sonda].remove_node_by_ref(node.next.next)
                                    else:
                                        node.next.next.data[1] += delta_min
                                    
                                    aloc = True
                                    node.data[2] += delta_min

                        # somando antecipação do prev e postergação do next, torna viável?
                        elif (delta_prev + delta_next >= delta_min):
                            # se, antecipando prev primeiro, contempla janela
                            if ((node.data[1]-delta_prev>=dados[22][projeto]) and (node.data[2]+delta_setup+(delta_min - delta_prev)<=dados[23][projeto])):
                                flag_realoc = 'viavel'
                                # se há mudança de setup
                                if (delta_setup < 0):
                                    # checar viabilidade de diminuir release do next
                                    if (node.next.data[1] + delta_setup < dados[22][node.next.data[0]]):
                                        flag_realoc = 'inviavel'
                                if (flag_realoc == 'viavel'):
                                    # fazer antecipação do prev primeiro
                                    node.prev.data[1] -= delta_prev
                                    node.prev.data[2] -= delta_prev
                                    
                                    if (node.prev.prev.data[2] - delta_prev < node.prev.prev.data[1]):
                                        s[sonda].remove_node_by_ref(node.prev.prev)
                                    else:
                                        node.prev.prev.data[2] -= delta_prev
                                    
                                    node.data[1] -= delta_prev
                                    # fazer postergação do next depois
                                    node.next.data[1] += (delta_min - delta_prev)
                                    node.next.data[2] += (delta_min - delta_prev)
                                    
                                    if (node.next.next.data[1] + (delta_min - delta_prev) > node.next.next.data[2]):
                                        s[sonda].remove_node_by_ref(node.next.next)
                                    else:
                                        node.next.next.data[1] += (delta_min - delta_prev)
                                    
                                    aloc = True
                                    node.data[2] += (delta_min - delta_prev)
                            # senão, se, postergando nexto primeiro, contempla janela
                            elif ((node.data[1]-(delta_min - delta_next)>=dados[22][projeto]) and (node.data[2]+delta_setup+delta_next<=dados[23][projeto])):
                                flag_realoc = 'viavel'
                                # se há mudança de setup
                                if (delta_setup < 0):
                                    # checar viabilidade de diminuir release do next
                                    if (node.next.data[1] + delta_setup < dados[22][node.next.data[0]]):
                                        flag_realoc = 'inviavel'
                                if (flag_realoc == 'viavel'):
                                    # fazer postergação do next primeiro
                                    node.next.data[1] += delta_next
                                    node.next.data[2] += delta_next
                                    
                                    if (node.next.next.data[1] + delta_next > node.next.next.data[2]):
                                        s[sonda].remove_node_by_ref(node.next.next)
                                    else:
                                        node.next.next.data[1] += delta_next
                                    
                                    node.data[2] += delta_next
                                    # fazer antecipação do prev depois
                                    node.prev.data[1] -= (delta_min - delta_next)
                                    node.prev.data[2] -= (delta_min - delta_next)
                                    
                                    if (node.prev.prev.data[2] - (delta_min - delta_next) < node.prev.prev.data[1]):
                                        s[sonda].remove_node_by_ref(node.prev.prev)
                                    else:
                                        node.prev.prev.data[2] -= (delta_min - delta_next)
                                    
                                    aloc = True
                                    node.data[1] -= (delta_min - delta_next)
                            
                    # alocar projeto
                    if aloc:
                        
                        #print ('realocando: projeto ', projeto, ' sendo inserido na sonda', sonda, 
                        #       ' na janela de ', node.data[1], ' até ', node.data[2], 'com delta_min=', delta_min,
                        #       ' delta_prev=', delta_prev, ' delta_next=', delta_next, ' e delta_setup', delta_setup,
                        #       ', sendo setup=', setup, ', e proc=', proc)
                        
                        # atualiza release do next e due da janela, caso o setup tenha sido alterado
                        if delta_setup != 0:
                            node.next.data[1] += delta_setup
                            node.data[2] += delta_setup
                        
                        node.data[0] = projeto
                        
                        #PrintSolution(s)
                        #VerificaRestricoes(s, dados, fitness, gastos, desloc)
                        
                        proj_usados.add(copy.copy(projeto))
                        fitness += dados[19][projeto]
                        break
        
        # se projeto foi alocado, atualizar dados
        if (aloc):
            gastos += dados[20][projeto]
            # se sonda ainda tem candidatos, colocar de volta na heap com valor atualizado
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, ( (tam_prev + (final-inicio+1)) , sonda))
        # senão, apenas recolocar a sonda
        else:
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
    
    return fitness, gastos, s, s_candidatos

In [ ]:
def VerificaFuncaoObjetivo(s, valor, dados):
    
    """
    Documentação:
    """
    
    fitness = 0
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node)
            if node.data[0] != -1:
                fitness += dados[19][node.data[0]]
    
    #if round(fitness, 5) == round(valor, 5):
    #    print ('função objetivo correta')
    #else:
    #    print ('função objetivo incorreta: ', valor, ' reportado, e ', fitness, ' apurado')
    
    return round(fitness, 5) == round(valor, 5)

In [ ]:
def VerificaRestricaoOrcamento(s, valor, dados):
    
    """
    Documentação:
    """
    
    custo = 0
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node)
            if node.data[0] != -1:
                custo += dados[20][node.data[0]]
    
    #if round(custo, 5) == round(valor, 5):
    #    print ('orçamento correto')
    #else:
    #    print ('orçamento incorreto: ', valor, ' reportado, e ', custo, ' apurado')
    
    return round(custo, 5) == round(valor, 5)

In [ ]:
def VerificaRestricaoUnicidadeProjetos(s, dados):
    
    """
    Documentação:
    """
    
    result = True
    projetos = set()
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node)
            if node.data[0] != -1:
                if node.data[0] in projetos:
                    result = False
                    #print ('unicidade incorreta: projeto ', node.data[0], ' duplicado')
                    return result
                else:
                    projetos.add(node.data[0])
    
    #print ('unicidade correta')
    
    return result

In [ ]:
def VerificaRestricaoTimeWindows(s, dados):
    
    """
    Documentação:
    """
    
    result = True
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node)
            if node.data[0] != -1:
                if not ((node.data[1] >= dados[22][node.data[0]])  and (node.data[1] <= dados[23][node.data[0]])):
                    result = False
                    #print ('janelas de tempo incorretas: projeto ', node.data[0], ' tem contrato de ', 
                    #      dados[22][node.data[0]], ' a ', dados[23][node.data[0]], ' e está alocado de ', 
                    #      node.data[1], ' a ', node.data[2])
                    #return result
    
    #print ('janelas de tempo corretas')
    
    return result

In [ ]:
def VerificaRestricaoScheduling(s, dados, desloc):
    
    """
    Documentação:
    """
    
    lag = len(s.keys())
    
    result = True
    for sonda in s.keys():
        for node in s[sonda]:
            #print ('verificando sonda ', sonda, ' e node ', node.data[0], ' node_prev', node.prev)
            if node.data[0] != -1:
                
                # se projeto vem após origem da sonda
                if (node.prev == None):
                    setup = desloc[sonda][node.data[0]+lag]
                elif ( (node.prev.data[0] == -1) and (node.prev.prev == None) ):
                    setup = desloc[sonda][node.data[0]+lag]
                # senão, projeto vem após algum outro projeto
                else:
                    # se existe janela de sonda antes do projeto
                    if node.prev.data[0] == -1:
                        setup = desloc[node.prev.prev.data[0]+lag][node.data[0]+lag]
                    # senão, ele vem imediatamente após o outro projeto
                    else:
                        setup = desloc[node.prev.data[0]+lag][node.data[0]+lag]
                
                if (node.data[2]-node.data[1] + 1 != setup + dados[21][node.data[0]]):
                    result = False
                    #print ('setup incorreto: projeto ', node.data[0], ' alocado de ', 
                    #       node.data[1], ' a ', node.data[2], ', sendo setup=', setup, ' e proc. time=', 
                    #      dados[21][node.data[0]], ' logo, ', node.data[2]-node.data[1]+1, ' != ', 
                    #      setup + dados[21][node.data[0]])
                    return result
    
    #print ('setup correto')
    
    return result

In [ ]:
def VerificaRestricaoContinuidade(s):
    
    """
    Documentação:
    """
    
    result = True
    for sonda in s.keys():
        for node in s[sonda]:
            
            if (node.data[2] < node.data[1]):
                result = False
                #print ("Continuidade incorreta: tempo de início maior que tempo final")
                return result
            
            if (node.next != None):
                if (node.next.data[1] != node.data[2] + 1):
                    result = False
                    #print ("Continuidade incorreta: sequência não está na ordem de tempo")
                    return result
    
    #print ("Continuidade correta")
    
    return result

In [ ]:
def VerificaRestricoes(s, dados, valor_fit, valor_custos, desloc):
    
    """
    Documentação:
    """
    
    fo = VerificaFuncaoObjetivo(s, valor_fit, dados)
    
    orc = VerificaRestricaoOrcamento(s, valor_custos, dados)
    
    unic = VerificaRestricaoUnicidadeProjetos(s, dados)
    
    time_wind = VerificaRestricaoTimeWindows(s, dados)
    
    sched = VerificaRestricaoScheduling(s, dados, desloc)
    
    cont = VerificaRestricaoContinuidade(s)
    
    return fo & orc & unic & time_wind & sched & cont

In [ ]:
def RodarHeuristica(arquivo, criterio='lucro/custo', alpha=0.5, print_sol=True):
    
    """
    Documentação:
    """
    
    import CPUtimer
    import copy
    
    n_projetos, n_sondas, n_periodos, t_init, t_final, delta_t, custo_total, dados, desloc = Read_data(path=arquivo)
    
    timer = CPUtimer.CPUTimer()
    timer.reset()
    timer.start()
    
    f, gastos , s, _ = ConstruirSolucao(dados, n_projetos, n_sondas, n_periodos, custo_total, desloc, 
                                        criterio=criterio, alpha=alpha)
    
    check = VerificaRestricoes(s, dados, f, gastos, desloc)
    
    tempo_total = timer.get_time(reference = "total", unit = "seconds")
    tempo_medio = timer.get_time(reference = "average", unit = "seconds")
    
    if check:
        check_str = 'feasible'
    else:
        check_str = 'not feasible'
    
    if print_sol:
        print (check_str)
        print ('fitness: ', f)
        print ('gastos: ', gastos)
        print ('tempo total de execução (segundos) : ', tempo_total)
        print ('tempo médio de execução (segundos) ', tempo_medio)
        for i in range(n_sondas):
            for node in s[i]:
                print ('sonda ', i, ' executa projetos: ', node.data)
    
    return f, gastos, s, tempo_total, tempo_medio, check_str

In [ ]:
def RodarHeuristicaSensibilidade(diretorio, criterios=None, alpha=0.5):
    
    """
    Documentação:
    """
    
    from os import walk
    import copy
    import pandas as pd
    
    if criterios == None:
        criterios = ['lucro', 'lucro/custo', 'lucro/duracao', 'custo', 'duracao', 'janela']
    
    resultados = {}
    resultados['Instancia'] = []
    resultados['Viabilidade'] = []
    resultados['Criterio guloso'] = []
    resultados['Tempo total (s)'] = []
    resultados['Tempo medio (s)'] = []
    resultados['Valor da função objetivo'] = []
    
    arquivos = _, _, arquivos = next(walk(diretorio))
    arquivos = ["./instancias/" + f for f in arquivos]
    
    for f in arquivos:
        
        print ('processando arquivo: ', f)
        
        for c in criterios:
            
            print ('Utilizando critério ', c)
            
            fitness, gastos, s, t_total, t_medio, feasible = RodarHeuristica(arquivo=f, criterio=c, 
                                                                             alpha=alpha, print_sol=False)
            
            resultados['Instancia'].append(copy.deepcopy(f.split(diretorio)[1]))
            resultados['Viabilidade'].append(copy.deepcopy(feasible))
            resultados['Criterio guloso'].append(copy.deepcopy(c))
            resultados['Tempo total (s)'].append(copy.deepcopy(t_total))
            resultados['Tempo medio (s)'].append(copy.deepcopy(t_medio))
            resultados['Valor da função objetivo'].append(copy.deepcopy(fitness))
    
    df = pd.DataFrame(resultados)
    
    return resultados, df

In [ ]:
def MultStart_Heuristic(arquivo, n_iter=100, criterios=None, alpha=0.5, print_sol=True):
    
    """
    Documentação:
    """
    
    import CPUtimer
    import copy
    import numpy as np
    
    if criterios == None:
        criterios = ['lucro', 'lucro/custo', 'lucro/duracao', 'custo', 'duracao', 'janela']
    
    n_projetos, n_sondas, n_periodos, t_init, t_final, delta_t, custo_total, dados, desloc = Read_data(path=arquivo)
    
    criterio = np.random.choice(criterios)
    
    f, gastos , s, _ = ConstruirSolucao(dados, n_projetos, n_sondas, n_periodos, custo_total, desloc, 
                                        criterio=criterio, alpha=alpha)
    
    best_f = copy.copy(f)
    best_gastos = copy.copy(gastos)
    best_s = copy.copy(s)
    
    timer = CPUtimer.CPUTimer()
    timer.reset()
    timer.start()
    
    i = 1
    while(n_iter > i):
        
        i += 1
        
        criterio = np.random.choice(criterios)
        
        f, gastos , s, _ = ConstruirSolucao(dados, n_projetos, n_sondas, n_periodos, custo_total, desloc, 
                                            criterio=criterio, alpha=alpha)
        
        if (f > best_f):
            best_f = copy.copy(f)
            best_gastos = copy.copy(gastos)
            best_s = copy.copy(s)
    
    check = VerificaRestricoes(best_s, dados, best_f, best_gastos, desloc)
    
    tempo_total = timer.get_time(reference = "total", unit = "seconds")
    tempo_medio = timer.get_time(reference = "average", unit = "seconds")
    
    if check:
        check_str = 'feasible'
    else:
        check_str = 'not feasible'
    
    if print_sol:
        print (check_str)
        print ('fitness: ', best_f)
        print ('gastos: ', best_gastos)
        print ('tempo total de execução (segundos) : ', tempo_total)
        print ('tempo médio de execução (segundos) ', tempo_medio)
        for i in range(n_sondas):
            for node in best_s[i]:
                print ('sonda ', i, ' executa projetos: ', node.data)
    
    return best_f, best_gastos, best_s, tempo_total, tempo_medio, check_str

In [ ]:
def RodarMultStartVariosArquivos(diretorio, criterios=None, alpha=0.5):
    
    """
    Documentação:
    """
    
    from os import walk
    import copy
    import pandas as pd
    
    resultados = {}
    resultados['Instancia'] = []
    resultados['Viabilidade'] = []
    resultados['Tempo total (s)'] = []
    resultados['Tempo medio (s)'] = []
    resultados['Valor da função objetivo'] = []
    
    arquivos = _, _, arquivos = next(walk(diretorio))
    arquivos = ["./instancias/" + f for f in arquivos]
    
    for f in arquivos:
        
        print ('processando arquivo: ', f)
        
        fitness, gastos, s, t_total, t_medio, feasible = MultStart_Heuristic(arquivo=f, n_iter=100,
                                                                             criterios=criterios, 
                                                                             alpha=0.5, print_sol=False)
        
        resultados['Instancia'].append(copy.deepcopy(f.split(diretorio)[1]))
        resultados['Viabilidade'].append(copy.deepcopy(feasible))
        resultados['Tempo total (s)'].append(copy.deepcopy(t_total))
        resultados['Tempo medio (s)'].append(copy.deepcopy(t_medio))
        resultados['Valor da função objetivo'].append(copy.deepcopy(fitness))
    
    df = pd.DataFrame(resultados)
    
    return resultados, df

### Rodando em algumas instâncias

In [ ]:
ls instancias/

In [ ]:
arq = 'instancias/instancia_50projetos_2sondas_delta_t28.dat'

### Uma instâncias específica com diferentes critérios e algoritmos

In [ ]:
%%time

c = 'lucro'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, alpha=0.5, print_sol=True)

In [ ]:
%%time

c = 'lucro/custo'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, alpha=0.5, print_sol=True)

In [ ]:
%%time

c = 'lucro/duracao'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, alpha=0.5, print_sol=True)

In [ ]:
%%time

c = 'custo'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, alpha=0.5, print_sol=True)

In [ ]:
%%time

c = 'duracao'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, alpha=0.5, print_sol=True)

In [ ]:
%%time

c = 'janela'

f, gastos, s, tempo, t_medio, feas = RodarHeuristica(arquivo=arq, criterio=c, alpha=0.5, print_sol=True)

In [ ]:
%%time

f, gastos, s, tempo, t_medio, feas = MultStart_Heuristic(arquivo=arq, n_iter=100, criterios=None, 
                                                         alpha=0.5, print_sol=True)

### Rodar para todos os arquivos

In [ ]:
%%time

d = "./instancias"

resultados, df_resultados_h = RodarHeuristicaSensibilidade(diretorio=d, criterios=None, alpha=0.999)

In [ ]:
df_resultados_h

In [ ]:
df_resultados_h['Viabilidade'].value_counts()

In [ ]:
df_resultados_h[df_resultados_h['Viabilidade']=='not feasible']

In [ ]:
%%time

d = "./instancias"

resultados_ms, df_resultados_ms = RodarMultStartVariosArquivos(diretorio=d, criterios=None, alpha=0.5)

In [ ]:
df_resultados_ms

In [ ]:
df_resultados_ms['Viabilidade'].value_counts()

In [ ]:
df_resultados_ms[df_resultados_ms['Viabilidade']=='not feasible']

In [ ]:
df_resultados = df_resultados_h.merge(df_resultados_ms, how='inner', on='Instancia', suffixes=['_H', '_MS'])

In [ ]:
df_resultados

In [ ]:
df_resultados['delta (%)'] = (df_resultados['Valor da função objetivo_MS'] - df_resultados['Valor da função objetivo_H']) / df_resultados['Valor da função objetivo_H']

In [ ]:
df_resultados['delta (%)'].hist()

### Resultados exatos MIP

In [ ]:
resultados_mip = {}
resultados_mip['Instancia'] = []
resultados_mip['Modelo MIP'] = []
resultados_mip['Tempo (s)'] = []
resultados_mip['Solução ótima'] = []

resultados_mip['Instancia'].append('/instancia_10projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(0.12)
resultados_mip['Solução ótima'].append(6089.95)

resultados_mip['Instancia'].append('/instancia_10projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('CTP')
resultados_mip['Tempo (s)'].append(0.19)
resultados_mip['Solução ótima'].append(6089.95)

resultados_mip['Instancia'].append('/instancia_20projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(1.16)
resultados_mip['Solução ótima'].append(8235.27)

resultados_mip['Instancia'].append('/instancia_20projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('CTP')
resultados_mip['Tempo (s)'].append(0.12)
resultados_mip['Solução ótima'].append(8235.27)

resultados_mip['Instancia'].append('/instancia_30projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(7.22)
resultados_mip['Solução ótima'].append(4699.8)

resultados_mip['Instancia'].append('/instancia_30projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('CTP')
resultados_mip['Tempo (s)'].append(0.85)
resultados_mip['Solução ótima'].append(4699.8)

resultados_mip['Instancia'].append('/instancia_40projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(4.48)
resultados_mip['Solução ótima'].append(14477)

resultados_mip['Instancia'].append('/instancia_50projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(3.94)
resultados_mip['Solução ótima'].append(18389.9)

resultados_mip['Instancia'].append('/instancia_60projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(31.81)
resultados_mip['Solução ótima'].append(13724)

resultados_mip['Instancia'].append('/instancia_70projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(1455.17)
resultados_mip['Solução ótima'].append(26644.3)

resultados_mip['Instancia'].append('/instancia_80projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(151.24)
resultados_mip['Solução ótima'].append(20310.7)

resultados_mip['Instancia'].append('/instancia_90projetos_2sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(13640.24)
resultados_mip['Solução ótima'].append(30071.4)

resultados_mip['Instancia'].append('/instancia_100projetos_5sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(174.57)
resultados_mip['Solução ótima'].append(58455.8)

resultados_mip['Instancia'].append('/instancia_100projetos_10sondas_delta_t28.dat')
resultados_mip['Modelo MIP'].append('TI')
resultados_mip['Tempo (s)'].append(371.33)
resultados_mip['Solução ótima'].append(32790.6)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_resultados_mip = pd.DataFrame(resultados_mip)

In [ ]:
df_resultados_mip

### Comparando heurísticas com MIP

In [ ]:
df_resultados_net = df_resultados.merge(df_resultados_mip, how='inner', on='Instancia').copy()

In [ ]:
df_resultados_net

In [ ]:
df_resultados_net['GAP MultiStart'] = round(df_resultados_net['Solução ótima'] - df_resultados_net['Valor da função objetivo_MS'], 2)
df_resultados_net['GAP MultiStart (%)'] = round(df_resultados_net['Solução ótima'] - df_resultados_net['Valor da função objetivo_MS'], 2) / df_resultados_net['Solução ótima']

In [ ]:
df_resultados_net

In [ ]:
df_resultados_net['GAP MultiStart'].unique()

In [ ]:
df_resultados_net['GAP MultiStart'].hist()

In [ ]:
df_resultados_net['GAP MultiStart (%)'].unique()

In [ ]:
df_resultados_net['GAP MultiStart (%)'].value_counts()

In [ ]:
df_resultados_net['GAP MultiStart (%)'].hist()

In [ ]:
df_resultados_net[df_resultados_net['GAP MultiStart (%)'] < 0]['Instancia'].unique()

In [ ]:
df_resultados_net.loc[df_resultados_net['Instancia']=='/instancia_50projetos_2sondas_delta_t28.dat']

### Exportar resultados

# TODO

df_resultados.to_csv(path_or_buf='resultados.csv', header=True, index=False, decimal='.')